## Etape 1: récupérez les données et commencez à les regarder 

1. Créez une base sqlite localement
2. Créez deux tables et uploadez les données
3. Récupérez les tables sous forme de dataframe python et faites une exploration rapide de ces données.
4. Quels sont selon vous dans les données patients les données personnelles (identificateurs & quasi-identificateurs) et les données sensibles ?
5. De combien de patients retrouvez vous le nom en faisant une jointure entre ces deux tables ?

### Soluce

In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import sqlite3
from datetime import date
from dateutil.relativedelta import relativedelta

import numpy as np
import pandas as pd

import pydp as dp  # by convention our package is to be imported as dp (for Differential Privacy!)
from pydp.algorithms.laplacian import BoundedMean, Count

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as patches



from anonymity.mondrian import MondrianAnonymizer, is_k_anonymous
from anonymity.plot import plot_mondrian

sns.set_style("whitegrid")

1. Créez une base sqlite localement

In [ ]:
db_filepath = Path('tp_anonymisation.db')
db_filepath.unlink(missing_ok=True) # rm db if exists

In [ ]:
conn = sqlite3.connect(db_filepath)

2. Créez deux tables et uploadez les données

In [ ]:
data_dir = Path('../data/')
items_to_feed = [(data_dir / 'personal_data.csv', 'personal_data'),
                 (data_dir / 'sensitive_data.csv', 'sensitive_data')]

for filepath, table_name in items_to_feed:
    pd.read_csv(filepath, sep=';')\
        .to_sql(table_name, conn, if_exists='replace', index=False)

Récupérez les tables sous forme de dataframe python et faites une exploration rapide de ces données.

In [ ]:
q = '''
select *
from sensitive_data
'''
df_patients = pd.read_sql(q, conn)

Nombre de patients dans notre fichier de données:

In [ ]:
len(df_patients)

In [ ]:
df_patients.head()

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(10, 12))

# age distribution
ax00 = ax[0, 0]
df_patients['age'].hist(ax=ax00)
ax00.set_xlabel('age')

# sex "ratio"
ax01 = ax[0, 1]
df_patients['sexe'].value_counts().plot(ax=ax01, kind='bar')

# repartition selon le département
ax10 = ax[1, 0]
df_patients['code_postal'].apply(lambda x: str(x)[:2]).value_counts().plot(ax=ax10, kind='bar')

# tension arterielle
ax11 = ax[1, 1]
df_patients['tension_arterielle'].value_counts().plot(ax=ax11, kind='barh')

# tension arterielle
ax21 = ax[2, 0]
df_patients['consommation_alcool'].value_counts().plot(ax=ax21, kind='barh')

# tension arterielle
ax22 = ax[2, 1]
df_patients['activite_physique'].value_counts().plot(ax=ax22, kind='barh')

fig.tight_layout()

In [ ]:
q = '''
select *
from personal_data
'''
df_identification = pd.read_sql(q, conn)

Nombre de personnes dans notre fichier de données marketting:

In [ ]:
len(df_identification)

In [ ]:
df_identification.head()

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(10, 8))

# age distribution
ax00 = ax[0, 0]
df_identification['age'].hist(ax=ax00)
ax00.set_xlabel('age')

# sex "ratio"
ax01 = ax[0, 1]
df_identification['sexe'].value_counts().plot(ax=ax01, kind='bar')

# repartition according to department
ax10 = ax[1, 0]
df_identification['code_postal'].apply(lambda x: str(x)[:2]).value_counts().plot(ax=ax10, kind='bar')

fig.tight_layout()

4. Quels sont selon vous dans les données patients les données personnelles (identificateurs & quasi-identificateurs) et les données sensibles ?
5. De combien de patients retrouvez vous le nom en faisant une jointure entre ces deux tables ?

In [ ]:
df_naive_join = pd.merge(
    df_identification,
    df_patients,
    on=('nom', 'prenom', 'sexe', 'age', 'adresse', 'code_postal')
)

Nombre de patients de notre fichier de données qui correspondent (nom, prenom, sexe, age, adresse, code_postal) à des personnes issues du fichier de données marketting:

In [ ]:
len(df_naive_join)

In [ ]:
df_naive_join.head()

Nombre de patients de notre fichier de données qui correspondent  (nom, prenom, sexe, age, adresse, code_postal) à des personnes issues du fichier de données marketting (sans ambiguité):

In [ ]:
len(df_naive_join) - df_naive_join.duplicated().sum()

## Etape 2: Pseudonymisez vos données puis attaquez les

1. Supprimer les identificateurs et les remplacer par une valeur unique par patient.
2. Chargez les données pseudonymisées. Combien d'individus vous arrivez à identifier de manière unique et pour combien retrouvez vous le nom et le prénom ?
    * Quelles sont selon vous les limites de la pseudonymisation ?
    * Que feriez vous pour améliorer cette approche ?
3. Mettez en oeuvre vos idées. Pour rappel, dans le cadre de votre étude, il vous faut conserver au minimum sujet des patients:
    * une idée de l'âge
    * le genre
    * une idée de la localisation géographique

### Soluce

1. Supprimer les identificateurs et les remplacer par une valeur unique par patient.

In [ ]:
df_patients_pseudo = df_patients.copy()
del df_patients_pseudo['nom']
del df_patients_pseudo['prenom']

In [ ]:
df_patients_pseudo.to_sql('sensitive_data_pseudo', conn, if_exists='replace', index=False)

2. Chargez les données pseudonymisées. Combien d'individus vous arrivez à identifier de manière unique et pour combien retrouvez vous le nom et le prénom ?
    * Quelles sont selon vous les limites de la pseudonymisation ?
    * Que feriez vous pour améliorer cette approche ?

In [ ]:
q = '''
select *
from sensitive_data_pseudo 
group by age, sexe, adresse
having count(*) == 1
'''

n_pseudo_unique = pd.read_sql(q, conn).shape[0]

In [ ]:
n_pseudo_unique

In [ ]:
q = '''
with all_matches as (
select pdata.nom,
       pdata.prenom,
       sdata.*
from sensitive_data_pseudo as sdata
inner join personal_data as pdata
on pdata.age = sdata.age and pdata.adresse = sdata.adresse and pdata.sexe = sdata.sexe
)
select *
from all_matches
group by age, adresse, sexe
having count(*) == 1
'''
df_pseudo_identified = pd.read_sql(q, conn)

Nombre de patients de notre fichier de données pseudonymisé qui correspondent (nom, prenom, adresse) à des personnes issues du fichier de données marketting (sans ambiguité):

In [ ]:
len(df_pseudo_identified)

In [ ]:
q = '''
with all_matches as (
select pdata.nom,
       pdata.prenom,
       sdata.*
from sensitive_data_pseudo as sdata
inner join personal_data as pdata
on pdata.age = sdata.age and pdata.adresse = sdata.adresse and pdata.sexe = sdata.sexe
),

ambiguous as (
select age, adresse, sexe
from all_matches
group by age, adresse, sexe
having count(*) > 1
)
select personal_data.*
from personal_data
join ambiguous
on personal_data.age = ambiguous.age and personal_data.sexe = ambiguous.sexe and personal_data.adresse = ambiguous.adresse
'''
df_pseudo_unidentified = pd.read_sql(q, conn)

In [ ]:
df_pseudo_unidentified.sort_values(['sexe', 'age', 'adresse'])

3. Mettez en oeuvre vos idées. Pour rappel, dans le cadre de votre étude, il vous faut conserver au minimum sujet des patients:
    * une idée de l'âge
    * le genre
    * une idée de la localisation géographique

In [ ]:
df_patients_anonym_naive = pd.read_sql('select * from sensitive_data_pseudo', conn)
s_age = df_patients_anonym_naive['age']
df_patients_anonym_naive['age_cut'] = pd.cut(s_age, range(0, 110, 10))
del df_patients_anonym_naive['date_naissance']
del df_patients_anonym_naive['adresse']
del df_patients_anonym_naive['age']

In [ ]:
df_patients_anonym_naive.head()

In [ ]:
df_identification_anonym_naive = pd.read_sql('select * from personal_data', conn)
s_age = df_identification_anonym_naive['age']
df_identification_anonym_naive['age_cut'] = pd.cut(s_age, range(0, 110, 10))

In [ ]:
mask = df_patients_anonym_naive.duplicated(['age_cut', 'code_postal', 'sexe'], keep=False)
df_patients_anonym_naive_identifiable = df_patients_anonym_naive[~mask]

Nombre de patients réidentifiables (identifiés de manière unique dans les données de santé) à partir de l'age (discretisé) du code postal et du sexe:

In [ ]:
len(df_patients_anonym_naive)

In [ ]:
df_merge = pd.merge(df_patients_anonym_naive,
                    df_identification_anonym_naive,
                    on=('age_cut', 'code_postal', 'sexe'),
                    how='inner')

Nombre de patients réidentifiés à partir de l'age (discretisé) du code postal et du sexe:

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))
s = df_merge.groupby(['age_cut', 'code_postal', 'sexe']).size()
s[(s>0) & (s <=10)].value_counts().sort_index().plot(kind='bar', ax=ax)

In [ ]:
df_merge_identified = df_merge[~df_merge.duplicated(['age_cut', 'code_postal', 'sexe'], keep=False)]

In [ ]:
df_merge_identified

## Anonymisation

1. Déterminez le paramètre K associé à votre anonymisation
2. Décrivez sur papier un algorithme simple pour automatiser la procédure de K-anonymisation (vous pouvez l'implémenter en bonus)
3. 4-anonymisez ces données (manuellement, à partir du package anonymity [(doc)](https://abel-betraoui.github.io/tp-insa-anonymity/anonymity.html#module-anonymity) ou de votre propre implémentation)
4. En vous servant des différentes limites de la K-anonymisation, tâchez de retrouver le plus d'informations possibles sur les patients du jeu de données K-anonymisé

### Soluce

1. Déterminez le paramètre K associé à votre anonymisation

In [ ]:
s = df_patients_anonym_naive.groupby(['sexe', 'code_postal', 'age_cut']).size()
s[s>0].min()

2. Décrivez sur papier un algorithme simple pour automatiser la procédure de K-anonymisation (vous pouvez l'implémenter en bonus)

<img src="images/pseudocode.png" width="300" alt="pseudocode">

K. LeFevre, D. J. DeWitt and R. Ramakrishnan, "Mondrian Multidimensional K-Anonymity," 22nd International Conference on Data Engineering (ICDE'06), 2006, pp. 25-25, doi: 10.1109/ICDE.2006.101.

2. 5-anonymisez ces données (manuellement ou pensez à un algorithme simple pour le faire)

In [ ]:
df_patients_anonym = df_patients.copy()
del df_patients_anonym['nom']
del df_patients_anonym['prenom']
del df_patients_anonym['adresse']

In [ ]:
d_metadata = {'age': {},
 'code_postal': {},
 'sexe': {'prevent_generalization': True},
 'activite_physique': {'is_sensitive': True},
 'consommation_alcool': {'is_sensitive': True},
 'tension_arterielle': {'is_sensitive': True}}

In [ ]:
anonymizer = MondrianAnonymizer(k=5)
df_anonymized = anonymizer.anonymize(df_patients_anonym, d_metadata)

In [ ]:
df_anonymized.head()

In [ ]:
df_anomymized_h = df_anonymized.query('sexe == 1')
df_anomymized_f = df_anonymized.query('sexe == 2')

In [ ]:
df_anomymized_h.groupby(['sexe', 'code_postal', 'age']).size().to_frame().rename(columns={0: 'size'}).min()

In [ ]:
df_anomymized_f.groupby(['sexe', 'code_postal', 'age']).size().to_frame().rename(columns={0: 'size'}).min()

In [ ]:
plot_mondrian(df_patients_anonym.query('sexe == 1'), df_anomymized_h)

In [ ]:
plot_mondrian(df_patients_anonym.query('sexe == 2'), df_anomymized_f)

3. En vous servant des différentes limites de la K-anonymisation, tâchez de retrouver le plus d'informations possibles sur les patients du jeu de données K-anonymisé

In [ ]:
def dummify_cols(df, cols):
    df_dummies = pd.get_dummies(df[cols])
    df_dummies.rename(columns={"activite_physique": "activite_physique_suffisante"}, inplace=True)
    df_dummies['activite_physique_insuffisante'] = 1 - df_dummies.activite_physique_suffisante

    dummy_cols = df_dummies.columns
    df_hm = pd.concat(
        [df[['sexe', 'code_postal', 'age']],
         df_dummies],
         axis=1)
    return df_hm.groupby(['sexe', 'code_postal', 'age'])[dummy_cols].mean()

In [ ]:
df_hm = dummify_cols(df_anonymized, anonymizer.sensitive)
fig, ax = plt.subplots(figsize=(20, 30))
sns.heatmap(df_hm[(df_hm == 1).any(axis=1)], ax=ax, annot=True)

In [ ]:
filepath_patients_to_find = data_dir / 'patients_to_find.csv'
df_patients_to_find = pd.read_csv(filepath_patients_to_find, sep=';')
l_patients = [s for _, s in df_patients_to_find.iterrows()]

display(df_patients_to_find)

for d_patient in l_patients:
    display(anonymizer.get_individual(df_anonymized, pd.Series(d_patient)))

6. Pouvez-vous répondre à ces questions ?
    * Combien de patients ont participé à l'étude ?
    * Combien de patients ont une tension artérielle très élevée ?
    * Parmi les patients de 60 ans et plus avec une consommation élevée d'alcool, quel pourcentage a
    une tension artérielle très élevée ?
    * Quel est le taux de tension artérielle "très élevée" à Villeurbanne (69100)?
    

Combien de patients ont participé à l'étude ?

In [ ]:
len(df_anonymized)

Combien de patients ont une tension artérielle très élevée ?

In [ ]:
len(df_anonymized.query('tension_arterielle == "très élevée"'))

Parmi les patients de 60 ans et plus avec une consommation élevée d'alcool, quel pourcentage à une tension artérielle très élevée ?

In [ ]:
df_anonymized

Impossible de répondre, les patients de 60 ans et plus ont été regroupés avec des patients de classe d'âges différentes.

In [ ]:
anonymizer.get_individual(df_anonymized, pd.Series({'age': 61}))

Impossible de répondre, les patients de Villeurbanne ont été regroupés avec des patients d'autres communes.

In [ ]:
anonymizer.get_individual(df_anonymized, pd.Series({'code_postal': 69100}))

## L-diversité

1. Utilisez le package anonymity pour comparer les comportements de la K-anonymisation et de la L-diversité. Qu'apporte la L-diversité, à quel(s) coût(s) ?

In [ ]:
anonymizer_kl = MondrianAnonymizer(k=5, l=2)
df_anonymized_kl = anonymizer_kl.anonymize(df_patients_anonym, d_metadata)
df_anonymized_kl_h = df_anonymized_kl.query('sexe == 1')
df_anonymized_kl_f = df_anonymized_kl.query('sexe == 2')

In [ ]:
df_hm = dummify_cols(df_anonymized_kl, anonymizer_kl.sensitive)
fig, ax = plt.subplots(figsize=(20, 30))
sns.heatmap(df_hm[(df_hm > .8).any(axis=1)], ax=ax, annot=True)

In [ ]:
plot_mondrian(df_patients_anonym.query('sexe == 1'), df_anonymized_kl_h)

In [ ]:
plot_mondrian(df_patients_anonym.query('sexe == 2'), df_anonymized_kl_f)

## Etape 4 - Créer le doute:  la confidentialité différentielle

* En vous inspirant de [ce notebook](https://github.com/OpenMined/PyDP/blob/dev/examples/Tutorial_3-Titanic_demo/titanic_notebook.ipynb), répondez aux questions suivantes et comparez les résultats obtenus **avec ou sans** confidentialité différentielle [(doc)](https://pydp.readthedocs.io/en/latest/pydp.html) ?
    * Combien de patients ont participé à l'étude ?
    * Combien de patients ont une tension artérielle très élevée ?
    * Parmi les patients de 60 ans et plus avec une consommation élevée d'alcool, quel pourcentage a une tension artérielle très élevée ?
    * Quel est le taux de tension artérielle "très élevée" à Villeurbanne (69100)?
    
* Utilisez la fonction result au lieu de quick result. 
    * Quelle est l'influence de epsilon sur le résultat final ?
    * Que se passe t-il lorsque le privacy budget est consommé ?
    * Pourquoi selon vous ?

Combien de patients ont participé à l'étude ?

In [ ]:
len(df_patients)

In [ ]:
privacy_budget = 1
count = Count(privacy_budget, dtype="float")
count.quick_result(df_patients.index)

In [ ]:
privacy_budget = 1
count = Count(privacy_budget, dtype="float")
count.add_entries(df_patients.index)
print(count.privacy_budget_left())
print(count.result(1))
print(count.privacy_budget_left())
print(count.result(1))

* Utilisez la fonction result au lieu de quick result. 
    * Quelle est l'influence de epsilon sur le résultat final ? *Plus epsilon est faible plus on se rapproche de la perfect privacy. C'est à dire que le risque est nul, les données obtenues sont par contre inexploitables.*
    * Que se passe t-il lorsque le privacy budget est consommé ? *On ne peut plus faire de requêtes.*
    * Pourquoi selon vous ? *Cela permet de limiter le risque de differential attacks*

Combien de patients ont une tension artérielle très élevée ?

In [ ]:
privacy_budget = 1
count = Count(privacy_budget, dtype="float")
count.quick_result(df_patients.index[df_patients.tension_arterielle == 'très élevée'])

Parmi les patients de 60 ans et plus avec une consommation élevée d'alcool, quel pourcentage à une tension artérielle très élevée ?

In [ ]:
privacy_budget = 1
mean = BoundedMean(privacy_budget, 0, 1, dtype="float")
dfx = df_patients[(df_patients['consommation_alcool'] == 'élevée') & (df_patients.age >= 60)]
s = (dfx.tension_arterielle == 'très élevée').astype(int).tolist()
mean.quick_result(s)

Les patients de villeurbanne ont ils un taux de tension artérielle très élevée plus grand que dans les autres communes?

In [ ]:
privacy_budget = 1
mean = BoundedMean(privacy_budget, 0, 1, dtype="float")
dfx = df_patients[(df_patients['code_postal'] == 69100)]
s = (dfx.tension_arterielle == 'très élevée').astype(int).tolist()
mean.quick_result(s)